In [1]:
!pip install -U setuptools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [2]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 9.9 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('wordnet')
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import pandas as pd
import numpy as np
import string
import random
import contractions
import spacy
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
with open(R'/content/drive/MyDrive/AI-ML/Natural Language Processing/Mini Project/Data/GL Bot.json') as corp:
    corpus = json.load(corp)

print(corpus)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for olympus', 'lot of p

In [5]:
intents = corpus['intents']

#1,2,3

In [ ]:
def string_cleaning(table,col):
    table[col] = table[col].apply(lambda x: re.sub(r'\d+','',x))
    print('removed numbers from text')
    table[col] = table[col].str.replace('[^A-Za-z\s]+', '')
    print('removed special characters')
    table[col] = table[col].str.replace('[^\w\s]+', '')
    print('removed puctuations')
    table[col] = table[col].str.strip()
    print('Additional whitespaces removed')
    print('--------------------------')
    return table[col].head()
def remove_stopwrds(a):
    for i in a:
        if i in stopwords.words('english'):
            a.pop(a.index(i))
        else:
            pass
    return a
def remove_nonascii(words):
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD',word).encode('ascii','ignore').decode('utf-8','ignore')
        new_words.append(new_word)
    return new_words
def lem_list(words):
    lemmatizer = WordNetLemmatizer()
    new_words = []
    for word in words:
        new_words.append(lemmatizer.lemmatize(word,pos='a'))
    return new_words
def joinwords(words):
    return ' '.join(words)
def ipcleaning(ip):
    cn_ip = re.sub('[^\w\s]+','',re.sub('[^A-Za-z\s]+','',re.sub(r'\d+','',ip))).strip()
    out2 = word_tokenize(contractions.fix(cn_ip))
    out3 = [out2.pop(out2.index(i)) for i in out2 if i not in stopwords.words('english')]
    out4 = remove_nonascii(out3)
    out5 = lem_list(out4)
    out6 = [i.lower() for i in out5]
    return out6
def simi(ip,crmat):
    b = []
    #temp = lambda i : ipcleaning(i)
    for i in crmat:
        z = ipcleaning(i)
        b = b + z
    vect = TfidfVectorizer()
    #vect.fit(c_crmat)
    crmat_vec = vect.fit_transform(b)
    ip_vec = vect.transform(ip)
    return cosine_similarity(ip_vec,crmat_vec).sum()
def chat():
    flag = True
    print('bot: Hello My Name is B')
    #name = input()
    while flag:
        userip = input()
        a_cl = ipcleaning(userip)
        pos = 0
        cos_vals = []
        cos_val = 0
        #while pos < 8: #or cos_val > 0.7:
        if 'break' in a_cl:
            flag = False
        elif len(a_cl)>0:
            for i in range(0,8) or cos_val > 0.7:
                cos_val = simi(a_cl,intents[i]['patterns'])
                cos_vals.append(cos_val)
                pos = i
                #print(pos)
            #print(cos_val,'|',pos,'|',cos_vals)
            max_pos = cos_vals.index(max(cos_vals,default=0))
            if max_pos == 1:
                print('bot: ',intents[max_pos]['responses'][0])
                flag = False
            else:
                print('bot: ',intents[max_pos]['responses'][0])
        else:
            print('bot: ',intents[0]['responses'][0])
    return print('Thank you')

In [ ]:
chat()

bot: Hello My Name is B
deep learning
bot:  Link: Neural Nets wiki
what is SL?
bot:  Link: Machine Learning wiki 
break
Thank you
